In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import tensorflow  as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *


from sklearn.metrics import mean_absolute_error
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from tensorflow.keras.callbacks import *
import os

2021-09-09 21:07:44.765970: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-09 21:07:44.766010: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [44]:
#Daten von einem Monat (08/2018) mit Wetter und echten Ausleih und Rückgabe-Werten
frame_alr = pd.read_csv(r'20152018/ausleihen_predicted.csv')

df_alr = pd.DataFrame(frame_alr)

df_alr2 = pd.DataFrame(df_alr)
df_alr.head()

,tempC,precipMM,humidity,windspeedKmph,year,day,holidays,workday,rueckgaben,ausleihen,...,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,ausleihenpredict
0,24,0.0,83,11,2018,1,0,1,2,2,...,0,0,0,0,1,0,0,0,0,1.208710
1,24,0.0,84,10,2018,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1.287797
2,24,0.3,86,9,2018,1,0,1,2,1,...,0,0,0,0,1,0,0,0,0,1.060808
3,24,0.4,87,8,2018,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0.921061
4,24,0.2,86,9,2018,1,0,1,0,2,...,0,0,0,0,1,0,0,0,0,1.016769


In [37]:
#Daten 2015-2018 Rückgaben mit Wetter
frame_rueckgaben = pd.read_csv(r'20152018/rueckgaben_mit_ausleihen_merged.csv')

df_rueckgaben = pd.DataFrame(frame_rueckgaben)
df_rueckgaben.head()

,year,month,day,hour,count,holidays,weekday,workday,tempC,precipMM,humidity,windspeedKmph,weatherDesc2,ausleihenpredict
0,2015,1,1,0,1,1,3,0,-3,0.0,52,15,Clear,0.000000
1,2015,1,1,2,1,1,3,0,-4,0.0,50,21,Clear,0.930812
2,2015,1,1,3,1,1,3,0,-4,0.0,49,24,Clear,0.000000
3,2015,1,1,10,2,1,3,0,-1,0.0,37,25,Sunny,3.069546
4,2015,1,1,11,2,1,3,0,0,0.0,36,24,Sunny,4.192753


In [38]:
df_rueckgaben = pd.get_dummies(df_rueckgaben, columns=['weekday', 'hour','weatherDesc2','month'] )
df_rueckgaben = df_rueckgaben.drop('day',axis=1)

In [39]:
df_rueckgaben.columns

Index(['year', 'count', 'holidays', 'workday', 'tempC', 'precipMM', 'humidity',
       'windspeedKmph', 'ausleihenpredict', 'weekday_0', 'weekday_1',
       'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6',
       'hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6',
       'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23',
       'weatherDesc2_Blizzard', 'weatherDesc2_Blowing snow',
       'weatherDesc2_Clear', 'weatherDesc2_Cloudy', 'weatherDesc2_Fog',
       'weatherDesc2_Freezing drizzle', 'weatherDesc2_Heavy rain',
       'weatherDesc2_Heavy rain at times', 'weatherDesc2_Heavy snow',
       'weatherDesc2_Ice pellets', 'weatherDesc2_Light drizzle',
       'weatherDesc2_Light freezing rain', 'weatherDesc2_Light rain',
       'weatherDesc2_Light rain shower', 'weatherDesc2_Light sleet',
       'weatherDesc2_L

In [45]:
df_alr = pd.DataFrame(df_alr, columns=['year', 'holidays', 'workday', 'tempC', 'precipMM', 'humidity',
       'windspeedKmph', 'ausleihenpredict', 'weekday_0', 'weekday_1',
       'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6',
       'hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6',
       'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23',
       'weatherDesc2_Blizzard', 'weatherDesc2_Blowing snow',
       'weatherDesc2_Clear', 'weatherDesc2_Cloudy', 'weatherDesc2_Fog',
       'weatherDesc2_Freezing drizzle', 'weatherDesc2_Heavy rain',
       'weatherDesc2_Heavy rain at times', 'weatherDesc2_Heavy snow',
       'weatherDesc2_Ice pellets', 'weatherDesc2_Light drizzle',
       'weatherDesc2_Light freezing rain', 'weatherDesc2_Light rain',
       'weatherDesc2_Light rain shower', 'weatherDesc2_Light sleet',
       'weatherDesc2_Light sleet showers', 'weatherDesc2_Light snow',
       'weatherDesc2_Light snow showers', 'weatherDesc2_Mist',
       'weatherDesc2_Moderate or heavy freezing rain',
       'weatherDesc2_Moderate or heavy rain shower',
       'weatherDesc2_Moderate or heavy rain with thunder',
       'weatherDesc2_Moderate or heavy sleet',
       'weatherDesc2_Moderate or heavy snow showers',
       'weatherDesc2_Moderate rain', 'weatherDesc2_Moderate rain at times',
       'weatherDesc2_Moderate snow', 'weatherDesc2_Overcast',
       'weatherDesc2_Partly cloudy',
       'weatherDesc2_Patchy freezing drizzle possible',
       'weatherDesc2_Patchy heavy snow', 'weatherDesc2_Patchy light drizzle',
       'weatherDesc2_Patchy light rain',
       'weatherDesc2_Patchy light rain with thunder',
       'weatherDesc2_Patchy light snow', 'weatherDesc2_Patchy moderate snow',
       'weatherDesc2_Patchy rain possible',
       'weatherDesc2_Patchy sleet possible',
       'weatherDesc2_Patchy snow possible', 'weatherDesc2_Sunny',
       'weatherDesc2_Thundery outbreaks possible',
       'weatherDesc2_Torrential rain shower', 'month_1', 'month_2', 'month_3',
       'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9',
       'month_10', 'month_11', 'month_12'])

In [35]:
#df_alr = pd.get_dummies(df_alr, columns=['weekday', 'hour','weatherDesc2','month'] )

In [43]:
df_alr.head()

,year,count,holidays,workday,tempC,precipMM,humidity,windspeedKmph,ausleihenpredict,weekday_0,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,2018,NaN,0,1,24,0.0,83,11,1.208710,0,...,0,0,0,0,0,1,0,0,0,0
1,2018,NaN,0,1,24,0.0,84,10,1.287797,0,...,0,0,0,0,0,1,0,0,0,0
2,2018,NaN,0,1,24,0.3,86,9,1.060808,0,...,0,0,0,0,0,1,0,0,0,0
3,2018,NaN,0,1,24,0.4,87,8,0.921061,0,...,0,0,0,0,0,1,0,0,0,0
4,2018,NaN,0,1,24,0.2,86,9,1.016769,0,...,0,0,0,0,0,1,0,0,0,0


In [28]:
df_rueckgaben.head()

,year,count,holidays,workday,tempC,precipMM,humidity,windspeedKmph,weekday_0,weekday_1,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,2015,1,1,0,-3,0.0,52,15,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2015,1,1,0,-4,0.0,50,21,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2015,1,1,0,-4,0.0,49,24,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2015,2,1,0,-1,0.0,37,25,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2015,2,1,0,0,0.0,36,24,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
df_alr = df_alr.drop(['day','rueckgaben','ausleihen'],axis=1)

In [ ]:
df_alr = df_alr.drop('weatherDesc2_Patchy freezing drizzle possible',axis = 1)

In [46]:
#Split Data
#y = df_alr['ausleihen']
#X = df_alr.drop('ausleihen',axis=1)

y = df_rueckgaben['count']
X = df_rueckgaben.drop('count',axis=1)

# Train-Test-Split
X_train , X_test, y_train, y_test = train_test_split(X,y,train_size=0.8,shuffle=False  )

In [47]:
ct = make_column_transformer(
 (MinMaxScaler(),["year","tempC","precipMM",'windspeedKmph',"humidity"]) , remainder="passthrough" #,
 )

#X_train[['year','tempC','precipMM','humidity']] = MinMaxScaler().fit_transform(X_train[['year','tempC','precipMM','humidity']])


ct.fit(X_train)

df_alr_normalized = ct.transform(df_alr)

#x_train_normalized = ct.transform(X_train)
#x_test_normalized = ct.transform(X_test)

In [149]:
x_test_normalized

array([[1.        , 0.41176471, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.41176471, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.41176471, 0.00724638, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.        , 0.45098039, 0.10144928, ..., 0.        , 0.        ,
        1.        ],
       [1.        , 0.45098039, 0.05072464, ..., 0.        , 0.        ,
        1.        ],
       [1.        , 0.47058824, 0.36231884, ..., 0.        , 0.        ,
        1.        ]])

In [48]:
model = keras.models.load_model('20152018/rueckgaben-modell1')

In [49]:
predicted_ausleihen = model.predict(df_alr_normalized)

In [50]:
df_alr2['rueckgabenpredict'] = predicted_ausleihen

In [51]:
df_alr2.to_csv('predicted_ausleihen_mit_rueckgaben_final.csv',index=False)

In [52]:
df_alr2

,tempC,precipMM,humidity,windspeedKmph,year,day,holidays,workday,rueckgaben,ausleihen,...,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,ausleihenpredict,rueckgabenpredict
0,24,0.0,83,11,2018,1,0,1,2,2,...,0,0,0,1,0,0,0,0,1.208710,1.801958
1,24,0.0,84,10,2018,1,0,1,0,0,...,0,0,0,1,0,0,0,0,1.287797,1.295098
2,24,0.3,86,9,2018,1,0,1,2,1,...,0,0,0,1,0,0,0,0,1.060808,1.001227
3,24,0.4,87,8,2018,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0.921061,0.961847
4,24,0.2,86,9,2018,1,0,1,0,2,...,0,0,0,1,0,0,0,0,1.016769,0.965510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,23,0.2,68,19,2018,31,0,1,14,15,...,0,0,0,1,0,0,0,0,25.805307,22.074966
740,22,0.0,69,19,2018,31,0,1,9,11,...,0,0,0,1,0,0,0,0,16.487340,15.755519
741,22,0.0,70,19,2018,31,0,1,8,7,...,0,0,0,1,0,0,0,0,9.240326,10.424086
742,22,0.0,71,18,2018,31,0,1,7,8,...,0,0,0,1,0,0,0,0,5.256702,5.545869


In [128]:
df_rueck = pd.read_csv(r'20152018/rueckgaben_mit_ausleihen_merged.csv')

In [135]:
df_rueck.head(10)

,year,month,day,hour,count,holidays,weekday,workday,tempC,precipMM,humidity,windspeedKmph,weatherDesc2,ausleihen
24232,2018,8,1,0,2,0,2,1,24,0.0,83,11,Overcast,2.222233
24233,2018,8,1,2,2,0,2,1,24,0.3,86,9,Light rain shower,0.906377
24234,2018,8,1,5,3,0,2,1,25,0.8,85,10,Light rain shower,1.049075
24235,2018,8,1,6,4,0,2,1,25,1.1,83,12,Light rain shower,3.686494
24236,2018,8,1,7,12,0,2,1,26,0.6,79,14,Light rain shower,17.914927
24237,2018,8,1,8,44,0,2,1,27,0.8,74,16,Light rain shower,33.040900
24238,2018,8,1,9,29,0,2,1,28,1.1,70,18,Light rain shower,23.247295
24239,2018,8,1,10,13,0,2,1,28,0.5,69,18,Light rain shower,11.091673
24240,2018,8,1,11,12,0,2,1,28,1.6,69,18,Moderate or heavy rain shower,9.366283
24241,2018,8,1,12,12,0,2,1,29,2.2,68,18,Moderate or heavy rain shower,11.161324


In [131]:
df_rueck = df_rueck[(df_rueck['year']== 2018) & (df_rueck['month'] == 8)]

In [143]:
df_rueck['ausleihen'].mean()

20.991987113750003